In [3]:
from helpers import stats, weapon_types, prof_translator, item_translator, proffesions
from game_structures import Item

In [4]:
import csv

# write to file serialized items
def csvSerializer(items : list, item_type : str, output_file : str):
    with open(output_file, "w", encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['lvl', 'prof', 'type', 'name', 'stats'])
        for el in items:
            # Split each item's data
            split_lst = el['data-stat'].split(';')
            # return & split by only those which contain '='
            params = [p.split('=') for p in split_lst if "=" in p]
            dct = {p[0] : p[1] for p in params}
            # add key for lack of proffesion requirement
            if 'reqp' not in dct:
                dct['reqp'] = 'wpbhtm'
            try:
                single_item = Item(int(dct['lvl']),
                    dct['reqp'], 
                    item_type, 
                    el['data-name'], 
                    {stats[p] : dct[p] for p in dct if p in stats})
                #print(single_item.toList())
                writer.writerow(single_item.toList())  
            except KeyError:
                print(f"KeyError -> Item name: {el['data-name']}; stats:{el['data-stat']}")


In [26]:
import requests
from bs4 import BeautifulSoup
from time import sleep

# Blade dancer url:
#https://margohelp.pl/przedmioty/b/1/4/TancerzOstrzy/Jednor%C4%99czne/Heroiczne/1-150/
# Paladin url:
#https://margohelp.pl/przedmioty/p/1/4/Paladyn/Jednor%C4%99czne/Heroiczne/1-150/
# Warrior url:
#https://margohelp.pl/przedmioty/w/1/4/Wojownik/Jednor%C4%99czne/Heroiczne/1-150/
# Mage url:
#https://margohelp.pl/przedmioty/m/6/4/Mag/R%C3%B3%C5%BCd%C5%BCki/Heroiczne/1-150/
# Tracker url:
#https://margohelp.pl/przedmioty/t/4/4/Tropiciel/Dystansowe/Heroiczne/1-150/
# Hunter url:
#https://margohelp.pl/przedmioty/h/4/4/%C5%81owca/Dystansowe/Heroiczne/1-150/

item_classes = {
    4 : 'Heroiczne', # "heroic" class
    8 : 'Legendarne' # "legendary" class
}

# Function for downloading each proffesion with their item type items in separate .csv file
# type as arg. "heroic" to download heroic&legendary or "legendary" for this specific class
def scrapeAndSaveData(item_class : str):
    class_marks = list(item_classes.items())
    if item_class == 'legendary':
        final_class = class_marks[1]
    else:
        final_class = class_marks[0]
    for key, vals in prof_translator.items():
        for idx, acc in enumerate(vals):
            page_items = []
            for i in (0, 150):
                sleep(.5)
                page = requests.get(f"https://margohelp.pl/przedmioty/{key[0]}/{acc}/{final_class[0]}/{proffesions[key[0]]}/{item_translator[acc]}/{final_class[1]}/{1+i}-{150+i}/")
                soup = BeautifulSoup(page.content, 'html.parser')
                page_items.extend(soup.find_all(class_='item'))
            csvSerializer(page_items, weapon_types[key][idx], f'items/{item_class}/{proffesions[key[0]]}_{item_translator[acc]}.csv')

In [27]:
# Download items;
# works with following folder paths:
# ./items/heroic
# ./items/legendary
scrapeAndSaveData('legendary')